In [ ]:
import eikon as ek
import pandas as pd
from datetime import datetime,date, timedelta
import os

import psycopg2 as pg

import warnings
warnings.filterwarnings('ignore')

In [ ]:
api_key = os.environ.get('eikon_key')
ek.set_app_key(api_key)

### pull min data

In [ ]:
t0 = datetime(2022,1,1)
t0

In [ ]:
ric = ['FLT.AX','NCM.AX', 'BHP.AX']

In [ ]:
df = ek.get_timeseries(
    ric,
    start_date= t0,
    end_date=t0 + timedelta(30), # limits
    interval='minute',
    calendar='tradingdays')

df

### Create a table for each ticker in the list

In [ ]:
conn = pg.connect(
    host='localhost',
    user='postgres',
    database='timeseries',
    password='password')

cur = conn.cursor()
conn.autocommit=True

In [ ]:
for i in ric:
    table_name = i.replace('.AX','')
    
    cur.execute('''drop table if exists {}'''.format(table_name))

    sql = '''CREATE TABLE IF NOT EXISTS {}(
    time TIMESTAMP,
    open DOUBLE PRECISION,
    high DOUBLE PRECISION,
    low DOUBLE PRECISION,
    close DOUBLE PRECISION,
    volume NUMERIC
    )'''.format(table_name)

    cur.execute(sql)
    print('Created table: {}'.format(table_name))

### Insert data into corresponding tables

In [ ]:
for ticker in ric:
    table_name = ticker.replace('.AX','')
    data = df[ticker]
    
    sql = '''INSERT INTO {}(time, open, high, low, close, volume) VALUES (%s, %s, %s, %s, %s, %s)'''.format(table_name)  
    
    for idx,row in data.dropna().iterrows():
        cur.execute(sql, (idx, row.OPEN, row.HIGH, row.LOW, row.CLOSE, row.VOLUME))
        print('inserted', table_name, idx)
        
    print('👍 Gucci!')

### Print out the tables which need updating

In [ ]:
today = datetime.today().date()


for i in ric:
    ticker = i.replace('.AX', '')
    max_date = pd.read_sql('select max(time) from {}'.format(ticker), conn).values[0][0]
    max_date = pd.to_datetime(max_date)
    print('Last updated {} on {}'.format(ticker, max_date),'| Today is', today)
    if max_date != today:
        print(ticker, 'needs updating!')

### **Update tables in database**

In [ ]:
today = datetime.today().date()


for i in ric:
    ticker = i.replace('.AX', '')
    max_date = pd.read_sql('select max(time) from {}'.format(ticker), conn).values[0][0]
    max_date = pd.to_datetime(max_date)

    while True:
        try:
            if max_date != today:
                new_data = ek.get_timeseries(
                    i,
                    start_date = str(max_date),
                    end_date=str(max_date+timedelta(days=5)),
                    interval='minute',
                    calendar='tradingdays')

                sql = '''INSERT INTO {}(time, open, high, low, close, volume) VALUES (%s, %s, %s, %s, %s, %s)'''.format(ticker)

                for idx,row in new_data[new_data.index > max_date].dropna().iterrows():
                    cur.execute(sql, (idx, row.OPEN, row.HIGH, row.LOW, row.CLOSE, row.VOLUME))
                    print('inserted', idx, ticker, '| Days remaining: {}'.format(str(today - new_data.index[-1].date()).date()))
                    max_date = (new_data.index[-1])

            else:
                print('Todays date: ', today)
                print('{} Last update: '.format(ticker), max_date)
                print('{} Data is up to date!'.format(ticker))
                break

        except:
            print('😢')
            break


        

# Close connection

In [ ]:
conn.close()

# Psql shell
`\l` -> View all of the defined databases on the server.

 

`\connect database_name` or `\c database_name` -> connect to db

 

`\dt`  -> Shows all tables in the current schema

---

Programmatically (or from the psql interface too, of course):

`SELECT * FROM pg_catalog.pg_tables;`
The system tables live in the pg_catalog database.




